In [2]:
# getting dependancies
import pandas as pd
import researchpy as rp
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats
import seaborn as sns
import matplotlib as plt

ModuleNotFoundError: No module named 'researchpy'

In [3]:
pip --user install researchpy

Note: you may need to restart the kernel to use updated packages.


'C:\Program' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# getting the data
df=pd.read_csv(r'C:\Users\BIMMY\Downloads\FinalData.csv')
df.head()

In [ ]:
# get count plot for the outcome
ax = sns.countplot(x='outcome', data=df)
plt.show()

In [ ]:
# logistic regression model
import statsmodels.api as sm 
# get independent variables
X=df[['AGEINMONTHS','DAYSADMITTED','HIV_exposed','Malnutrition']]

In [ ]:
X2=df[['HIV_exposed','Malnutrition']]

In [ ]:
X3=df[['HIV_exposed']]

In [ ]:
X4=df[['Malnutrition']]

In [ ]:
# to get intercept -- this is optional
# X = sm.add_constant(X)
# get response variables

In [ ]:
Y = df.outcome



In [ ]:
# fit the model with maximum likelihood function
model = sm.Logit(endog=Y, exog=X).fit()
# output message

In [ ]:
print(model.summary())

In [ ]:
# get odds ratio
np.exp(model.params)

In [ ]:
#The above results are the log odds which is not intuitive (straight forward) in it's interpretation.
#Let's convert this to odds ratio and interpret the model again.
#To convert the log odds coefficients and confidence intervals, one needs to take the exponential of the values.

model_odds = pd.DataFrame(np.exp(model.params), columns= ['OR'])
model_odds['z-value']= model.pvalues
model_odds[['2.5%', '97.5%']] = np.exp(model.conf_int())

model_odds

In [ ]:
model2 = sm.Logit(endog=Y, exog=X2).fit()

In [ ]:
model3 = sm.Logit(endog=Y, exog=X3).fit()

In [ ]:
model4 = sm.Logit(endog=Y, exog=X4).fit()

In [ ]:
print(model2.summary())

In [ ]:
print(model3.summary())

In [ ]:
print(model4.summary())

In [ ]:
model2_odds = pd.DataFrame(np.exp(model2.params), columns= ['OR'])
model2_odds['z-value']= model2.pvalues
model2_odds[['2.5%', '97.5%']] = np.exp(model2.conf_int())

model2_odds

In [ ]:
model3_odds = pd.DataFrame(np.exp(model3.params), columns= ['OR'])
model3_odds['z-value']= model3.pvalues
model3_odds[['2.5%', '97.5%']] = np.exp(model3.conf_int())

model3_odds

In [ ]:
model4_odds = pd.DataFrame(np.exp(model4.params), columns= ['OR'])
model4_odds['z-value']= model4.pvalues
model4_odds[['2.5%', '97.5%']] = np.exp(model4.conf_int())

model4_odds

Interpretaion

In [ ]:
#The p values for all independent variables are significant (p < 0.05) 
#and suggests that these variables are highly associated with the outcome.

In [ ]:
#The odds of dying decreases by a factor of 0.8 for every unit increase of a child's age in months;
#The odds of dying also decreases by a factor of 0.75 for every unit increase in the days admitted in hospital.
#likewise, the odds of dying increases by a factor of 6.9 if a child is HIV exposed, and 4.9 if a child is malnourished.



Prediction of test dataset using fitted model

In [ ]:
# get the predicted values for the test dataset [0, 1]
pred = model.predict(exog=df[['AGEINMONTHS','DAYSADMITTED','HIV_exposed','Malnutrition']])
pred.head()
# output

In [ ]:
pred2 = model2.predict(exog=df[['HIV_exposed','Malnutrition']])
pred2.head()

In [ ]:
accuracy_score(y_true=list(df['outcome']), y_pred=list(round(pred2)))

In [ ]:
# predicted values > 0.5 classified as dischsrged (1) and <= 0.05 as dead (0)
round(pred)

In [ ]:
# get confusion matrix and accuracy of the prediction
# note: there may be slightly different results if you use sklearn LogisticRegression method
from sklearn.metrics import accuracy_score, confusion_matrix
confusion_matrix(y_true=list(df['outcome']), y_pred=list(round(pred)))



In the confusion matrix, diagonal numbers (124 and 5) indicates the correct predictions [true negatives (TN) and true positives(TP)] for the discharged (0) and dead (1) outcomes for pneumonia.
The other numbers (1 and 11) indicates incorrect predictions [false positives (FP) and false negatives (FN)] respectively

In [ ]:
# fitted model accuracy
accuracy_score(y_true=list(df['outcome']), y_pred=list(round(pred)))
# output

The above model is 91% accurate in its prediction of death via pneumonia.

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
from bioinfokit.visuz import stat

fpr, tpr, thresholds = roc_curve(y_true=list(df['outcome']), y_score=list(pred))
auc = roc_auc_score(y_true=list(df['outcome']), y_score=list(pred))
# plot ROC
stat.roc(fpr=fpr, tpr=tpr, auc=auc, shade_auc=True, per_class=True, legendpos='upper center', legendanchor=(0.5, 1.08), legendcols=3)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
#define the predictor variables and the response variable
X = df[['AGEINMONTHS','DAYSADMITTED','HIV_exposed','Malnutrition']]
y = df['outcome']

#split the dataset into training (70%) and testing (30%) sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0) 

#instantiate the model
model = LogisticRegression()

#fit the model using the training data
model.fit(X_train,y_train)

In [ ]:
#use model to predict probability that given y value is 1
y_pred_proba = model.predict_proba(X_test)[::,1]

#calculate AUC of model
auc = metrics.roc_auc_score(y_test, y_pred_proba)

#print AUC score
print(auc)



In [ ]:
print(model.intercept_, model.coef_, model.score(X_train,y_train))


In [ ]:
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, model.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()